# Đề thi Giữa kỳ Hệ khuyến nghị 2025 (đề thi mẫu).

## Khoa học dữ liệu IUH. Thời gian làm bài: 90 phút.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

**Bài 1.** (*2.5 điểm*) Vấn đề về *cold-start*.

Cho file dữ liệu csv về 20 bài hát được 100 users rate trong tháng qua, điểm từ 1 đến 5. Cấu trúc file: *id user, id bài hát, điểm rating*.

1) Sử dụng kỹ thuật tổng hợp 2 thông tin: số rating kết hợp rating trung bình, hãy đề xuất ra top 5 bài hát nên nghe cho một user cold-start.  
2) Giả sử có một user đã nghe bài hát STT 1. Sử dụng kỹ thuật đếm số cặp bài hát được nghe bởi cùng user, hãy đề xuất ra top 5 bài hát mà user này có thể cũng muốn nghe theo xu hướng.  
3) Hỏi nếu có thêm 1 item mới đưa vào hệ thống thì ta nên xử lý thế nào? SV tham khảo thêm tại đây:  

https://en.wikipedia.org/wiki/Cold_start_(recommender_systems)

In [13]:
import pandas as pd

df = pd.read_csv('data.csv', names=['userId', 'songId', 'rating'])
df = df.dropna(subset=['userId', 'songId', 'rating'])
# Gom nhóm theo bài hát
agg = df.groupby('songId').agg(
    v=('rating', 'count'),
    R=('rating', 'mean')
).reset_index()

# Trung bình toàn bộ rating
C = agg['R'].mean()

# Ngưỡng tối thiểu (ví dụ: 90th percentile)
m = agg['v'].quantile(0.90)

print(f'C = {C:.2f}, m = {m:.0f}')

def weighted_rating(x, m=m, C=C):
    v = x['v']
    R = x['R']
    return (v/(v+m))*R + (m/(v+m))*C

agg['WR'] = agg.apply(weighted_rating, axis=1)

# Sắp xếp top 5
top5 = agg.sort_values('WR', ascending=False).head(5)
print(top5)

C = 2.99, m = 33
    songId   v         R        WR
17      18  18  3.555556  3.189951
14      15  31  3.354839  3.167028
6        7  14  3.571429  3.163620
9       10  16  3.500000  3.156955
3        4  19  3.368421  3.128724


In [11]:
# --- Chuẩn hóa ---
df['điểm rating'] = pd.to_numeric(df['điểm rating'], errors='coerce')
df = df.dropna(subset=['điểm rating'])
# 1️⃣ Tính Weighted Rating (theo công thức IMDb)

# Gom nhóm theo bài hát để lấy số lượt rating (v) và trung bình (R)
agg = df.groupby('id bài hát')['điểm rating'].agg(['count', 'mean']).reset_index()
agg = agg.rename(columns={'count': 'v', 'mean': 'R'})

# Tính C (điểm trung bình toàn bộ bài hát)
C = df['điểm rating'].mean()

# Tính m (ngưỡng tối thiểu, ví dụ: phần vị 60%)
m = agg['v'].quantile(0.90)

# Tính Weighted Rating
agg['WR'] = (agg['v'] / (agg['v'] + m)) * agg['R'] + (m / (agg['v'] + m)) * C

# Lấy Top 5 bài hát có WR cao nhất
top5_WR = agg.sort_values('WR', ascending=False).head(5)

print("\nTop 5 bài hát được đề xuất cho user cold-start (Weighted Rating):")
print(top5_WR)


Top 5 bài hát được đề xuất cho user cold-start (Weighted Rating):
    id bài hát   v         R        WR
17          18  18  3.555556  3.198626
14          15  31  3.354839  3.173943
6            7  14  3.571429  3.173031
9           10  16  3.500000  3.165983
3            4  19  3.368421  3.137232


In [5]:
baiHat = df.groupby('id bài hát').agg( soLuotBinhChon = ('điểm rating', 'count'), soDiemTrungBinh = ('điểm rating', 'mean'))
baiHat['score'] = baiHat['soDiemTrungBinh'] * np.log(baiHat['soLuotBinhChon'])
top5BaiHat = baiHat.sort_values('score')
print('top 5 bài hát nên nghe: \n',baiHat.tail(5))


top 5 bài hát nên nghe: 
             soLuotBinhChon  soDiemTrungBinh      score
id bài hát                                            
16                      16         3.375000   9.357487
17                      20         2.700000   8.088477
18                      18         3.555556  10.276877
19                      24         2.833333   9.004486
20                      11         2.727273   6.539714


In [5]:
#Giả sử có một user đã nghe bài hát STT 1. Sử dụng kỹ thuật đếm số cặp bài hát được nghe bởi cùng user, hãy đề xuất ra top 5 bài hát mà user này có thể cũng muốn nghe theo xu hướng.
baiHat1 = df[df['id bài hát']==1]
user_nghe_bai1 = baiHat1['id user'].unique()
user_nghe_bai1

array([ 8, 13, 16, 17, 20, 14, 31, 36, 38, 40, 41, 48, 57, 58, 60, 63, 69,
       71, 74, 80, 85, 96])

In [6]:
baiHatLienQuan = df[df['id user'].isin(user_nghe_bai1) & df['id bài hát'] != 1]
baiHatLienQuan  = baiHatLienQan.groupby('id bài hát').agg(soLuotNghe = ('id bài hát','count'))
baiHatLienQuan_top = baiHatLienQuan.sort_values('soLuotNghe',ascending=False)
baiHatLienQuan_top.head(5)

,soLuotNghe
id bài hát,
12,56
14,34
11,24
15,23
19,19


**Bài 2.** (*3 điểm*) Vấn đề xây dựng *content-based data*.

Sử dụng dataset movie_lens trong đó có file *movies.csv* mô tả tên films và các thể loại (một phim có thể thuộc nhiều thể loại).

1. Đọc dữ liệu và lọc lại 100 films có nhiều thể loại nhất.
2. Xét một film X trong đó, tính ra top 10 films tương đồng với film X này nhất bằng độ tương đồng Jaccard.
3. Cho một user Y đã xem được 5 bộ films $1,2,3,4,5$ trong danh sách 100 films đã liệt kê ở trên, hãy xây dựng một profile cơ bản cho user này và khuyến nghị ra top 10 films mà user Y này có thể muốn xem.

In [9]:
df = pd.read_csv('movie_lens/movies.csv')


In [10]:
df['theLoai'] = df['genres'].apply(lambda x : x.split('|'))
df['soTheLoai'] = df['theLoai'].apply(len)
df_100 = df.sort_values('soTheLoai',ascending=False).head(100)
df_100.reset_index()

,index,movieId,title,genres,theLoai,soTheLoai
0,7441,81132,Rubber (2010),Action|Adventure|Comedy|Crime|Drama|Film-Noir|...,"[Action, Adventure, Comedy, Crime, Drama, Film...",10
1,5556,26701,Patlabor: The Movie (Kidô keisatsu patorebâ: T...,Action|Animation|Crime|Drama|Film-Noir|Mystery...,"[Action, Animation, Crime, Drama, Film-Noir, M...",8
2,7170,71999,Aelita: The Queen of Mars (Aelita) (1924),Action|Adventure|Drama|Fantasy|Romance|Sci-Fi|...,"[Action, Adventure, Drama, Fantasy, Romance, S...",7
3,1390,1907,Mulan (1998),Adventure|Animation|Children|Comedy|Drama|Musi...,"[Adventure, Animation, Children, Comedy, Drama...",7
4,5819,32031,Robots (2005),Adventure|Animation|Children|Comedy|Fantasy|Sc...,"[Adventure, Animation, Children, Comedy, Fanta...",7
...,...,...,...,...,...,...
95,5640,27397,Joint Security Area (Gongdong gyeongbi guyeok ...,Crime|Drama|Mystery|Thriller|War,"[Crime, Drama, Mystery, Thriller, War]",5
96,5634,27328,Monday (2000),Action|Comedy|Crime|Fantasy|Thriller,"[Action, Comedy, Crime, Fantasy, Thriller]",5
97,5632,27317,Audition (Ôdishon) (1999),Drama|Horror|Mystery|Romance|Thriller,"[Drama, Horror, Mystery, Romance, Thriller]",5
98,5631,27311,Batman Beyond: Return of the Joker (2000),Action|Animation|Crime|Sci-Fi|Thriller,"[Action, Animation, Crime, Sci-Fi, Thriller]",5


In [11]:
#Xét một film X trong đó, tính ra top 10 films tương đồng với film X này nhất bằng độ tương đồng Jaccard.
#hàm giao và hợp chỉ viết được khi kiểu dữ liệu là set
df['theLoai'] = df['theLoai'].apply(set)
phim1 = df.iloc[0]

tap_tuongDong = []
for i,phim in df.iterrows():
    if(phim['movieId'] != phim1['movieId']):
        giao =len( phim1['theLoai'].intersection(phim['theLoai']))
        hop = len(phim1['theLoai'].union(phim['theLoai']))
        if hop > 0:
            tuongDong = giao/hop
        else:
            tuongDong = 0
        tap_tuongDong.append({
            'movieId' : phim['movieId'],
            'title' : phim['title'],
            'theLoai' : phim['theLoai'],
            'tuongDong' : tuongDong
        })
doTuongDong = pd.DataFrame(tap_tuongDong)
doTuongDong_top = doTuongDong.sort_values('tuongDong',ascending = False)
doTuongDong_top

,movieId,title,theLoai,tuongDong
2808,3754,"Adventures of Rocky and Bullwinkle, The (2000)","{Comedy, Children, Fantasy, Adventure, Animation}",1.0
9429,166461,Moana (2016),"{Comedy, Children, Fantasy, Adventure, Animation}",1.0
7759,91355,Asterix and the Vikings (Astérix et les Viking...,"{Comedy, Children, Fantasy, Adventure, Animation}",1.0
6947,65577,"Tale of Despereaux, The (2008)","{Comedy, Children, Fantasy, Adventure, Animation}",1.0
6485,53121,Shrek the Third (2007),"{Comedy, Children, Fantasy, Adventure, Animation}",1.0
...,...,...,...,...
32,36,Dead Man Walking (1995),"{Drama, Crime}",0.0
9673,183011,The Commuter (2018),"{Drama, Mystery, Crime, Thriller}",0.0
9675,183199,Quest (2017),{Documentary},0.0
9677,183295,Insidious: The Last Key (2018),"{Mystery, Horror, Thriller}",0.0


In [12]:
#Cho một user Y đã xem được 5 bộ films $1,2,3,4,5$ trong danh sách 100 films đã liệt kê ở trên,
#hãy xây dựng một profile cơ bản cho user này và khuyến nghị ra top 10 films mà user Y này có thể muốn xem.
phim_user = df_100.iloc[[0,1,2,3,4]]
user_profile = set()
for theLoai in phim_user['theLoai']:
    user_profile.update(theLoai)
print('các thể loại đã xem: ',user_profile)

#tuongdong jaccard
deXuat = []
for i,phim in df.iterrows():
    if(phim['movieId'] not in  phim_user['movieId']):
        giao =len( phim1['theLoai'].intersection(phim['theLoai']))
        hop = len(phim1['theLoai'].union(phim['theLoai']))
        if hop > 0:
            tuongDong = giao/hop
        else:
            tuongDong = 0
        deXuat.append({
            'movieId' : phim['movieId'],
            'title' : phim['title'],
            'theLoai' : phim['theLoai'],
            'tuongDong' : tuongDong
        })
df_deXuat = pd.DataFrame(deXuat)
df_deXuat.sort_values('tuongDong',ascending = False).head(10)

các thể loại đã xem:  {'Comedy', 'Musical', 'IMAX', 'Film-Noir', 'Drama', 'Sci-Fi', 'Children', 'Horror', 'Mystery', 'Western', 'Animation', 'Action', 'Thriller', 'Romance', 'Adventure', 'Fantasy', 'Crime'}


,movieId,title,theLoai,tuongDong
8217,103755,Turbo (2013),"{Comedy, Children, Fantasy, Adventure, Animation}",1.0
1705,2294,Antz (1998),"{Comedy, Children, Fantasy, Adventure, Animation}",1.0
3567,4886,"Monsters, Inc. (2001)","{Comedy, Children, Fantasy, Adventure, Animation}",1.0
6192,45074,"Wild, The (2006)","{Comedy, Children, Fantasy, Adventure, Animation}",1.0
9428,166461,Moana (2016),"{Comedy, Children, Fantasy, Adventure, Animation}",1.0
6946,65577,"Tale of Despereaux, The (2008)","{Comedy, Children, Fantasy, Adventure, Animation}",1.0
2808,3754,"Adventures of Rocky and Bullwinkle, The (2000)","{Comedy, Children, Fantasy, Adventure, Animation}",1.0
0,1,Toy Story (1995),"{Comedy, Children, Fantasy, Adventure, Animation}",1.0
2999,4016,"Emperor's New Groove, The (2000)","{Comedy, Children, Fantasy, Adventure, Animation}",1.0
7758,91355,Asterix and the Vikings (Astérix et les Viking...,"{Comedy, Children, Fantasy, Adventure, Animation}",1.0


**Bài 3.** (*2 điểm*) Vấn đề dự đoán *missing value* trong lọc cộng tác.

Cho một bảng *utility matrix* giữa 4 user và 4 item như bên dưới:

|Rating     | $I_1$    | $I_2$  | $I_3$   |  $I_4$  |
|----------|----------|--------|---------|---------|
| $U_1$    | 4     | 3     | ?    |   4 | 
| $U_2$ | 3     | 3      | 5     |   5    |
| $U_3$  | 5     | 4      |   3  |    4 |
| $U_4$|   ?  |   2    | 3 |   5|

1. Dự đoán hai missing rating $r_{U_1, I_3}$ và $r_{U_4, I_1}$ bằng cách dùng độ tương đồng Pearson.
2. Câu hỏi tương tự trên nhưng chuẩn hoá bảng theo dạng center và sau đó dùng độ tương đồng cosine. So sánh các kết quả thu được.

In [13]:
data = {
    'U1' : [4,3,np.nan,4],
    'U2' : [3,3,5,5],
    'U3' : [5,4,3,4],
    'U4' : [np.nan,2,3,5],
}
df = pd.DataFrame(data, index= ['I1','I2','I3','I4'])

df = df.T
df

,I1,I2,I3,I4
U1,4.0,3.0,NaN,4.0
U2,3.0,3.0,5.0,5.0
U3,5.0,4.0,3.0,4.0
U4,NaN,2.0,3.0,5.0


In [14]:
import numpy as np
import pandas as pd

# Ma trận đánh giá (utility matrix)
du_lieu = pd.DataFrame({
    "I1": [4, 3, 5, np.nan],
    "I2": [3, 3, 4, 2],
    "I3": [np.nan, 5, 3, 3],
    "I4": [4, 5, 4, 5]
}, index=["U1", "U2", "U3", "U4"])

print("Bảng đánh giá ban đầu:")
print(du_lieu)

# --- 1. Tính độ tương đồng Pearson giữa các user ---
tuong_dong_pearson = du_lieu.T.corr(method='pearson')
print("\nĐộ tương đồng Pearson giữa các user:")
print(tuong_dong_pearson)

# --- Hàm dự đoán rating dựa trên Pearson ---
def du_doan_pearson(user, item):
    nguoi_dung = du_lieu.loc[user]
    cac_user_khac = du_lieu.drop(index=user)
    
    diem_tuong_dong = tuong_dong_pearson.loc[user]
    tong_tu, tong_mau = 0, 0
    
    for nguoi_khac, hang in cac_user_khac.iterrows():
        if not np.isnan(hang[item]):
            tong_tu += diem_tuong_dong[nguoi_khac] * hang[item]
            tong_mau += abs(diem_tuong_dong[nguoi_khac])
    
    return tong_tu / tong_mau if tong_mau != 0 else np.nan

# Dự đoán 2 giá trị còn thiếu
r_U1_I3 = du_doan_pearson("U1", "I3")
r_U4_I1 = du_doan_pearson("U4", "I1")

print(f"\nDự đoán r_U1,I3 (Pearson): {r_U1_I3:.2f}")
print(f"Dự đoán r_U4,I1 (Pearson): {r_U4_I1:.2f}")

# --- 2. Dự đoán lại sau khi chuẩn hoá và dùng Cosine ---
# Chuẩn hoá dữ liệu theo dạng center
du_lieu_chuanhoa = du_lieu.sub(du_lieu.mean(axis=1), axis=0)


from sklearn.metrics.pairwise import cosine_similarity

# Tính độ tương đồng cosine giữa các user
tuong_dong_cosine = pd.DataFrame(
    cosine_similarity(du_lieu_chuanhoa.fillna(0)),  # thay NaN = 0 để tính cosine
    index=du_lieu_chuanhoa.index,
    columns=du_lieu_chuanhoa.index
)

# Hàm dự đoán Cosine (dùng dữ liệu chuẩn hoá)
def du_doan_cosine(user, item):
    nguoi_dung = du_lieu_chuanhoa.loc[user]
    cac_user_khac = du_lieu_chuanhoa.drop(index=user)
    
    diem_tuong_dong = tuong_dong_cosine.loc[user]
    tong_tu, tong_mau = 0, 0
    
    for nguoi_khac, hang in cac_user_khac.iterrows():
        if not np.isnan(du_lieu.loc[nguoi_khac, item]):
            tong_tu += diem_tuong_dong[nguoi_khac] * hang[item]
            tong_mau += abs(diem_tuong_dong[nguoi_khac])
    
    du_doan = du_lieu.mean(axis=1)[user] + (tong_tu / tong_mau if tong_mau != 0 else 0)
    return du_doan

r_U1_I3_cosine = du_doan_cosine("U1", "I3")
r_U4_I1_cosine = du_doan_cosine("U4", "I1")

print(f"\nDự đoán r_U1,I3 (Cosine): {r_U1_I3_cosine:.2f}")
print(f"Dự đoán r_U4,I1 (Cosine): {r_U4_I1_cosine:.2f}")


Bảng đánh giá ban đầu:
     I1  I2   I3  I4
U1  4.0   3  NaN   4
U2  3.0   3  5.0   5
U3  5.0   4  3.0   4
U4  NaN   2  3.0   5

Độ tương đồng Pearson giữa các user:
     U1        U2        U3        U4
U1  1.0  0.500000  0.500000  1.000000
U2  0.5  1.000000 -0.707107  0.755929
U3  0.5 -0.707107  1.000000  0.188982
U4  1.0  0.755929  0.188982  1.000000

Dự đoán r_U1,I3 (Pearson): 3.50
Dự đoán r_U4,I1 (Pearson): 3.71

Dự đoán r_U1,I3 (Cosine): 3.57
Dự đoán r_U4,I1 (Cosine): 3.18


**Bài 4.** (*2.5 điểm*) Vấn đề thuật toán lọc cộng tác ở dạng *user-based* và *item-based*.

Sử dụng lại dataset ở bài 1 hoặc dataset *movie_lens* và chọn một thuật toán *collaborative filtering* đã chuẩn bị sẵn, thực hiện hai yêu cầu:
1) In ra ma trận tương đồng giữa các user và giữa các item sử dụng *sklearn.metrics.pairwise*.  

2) Duyệt theo từng user và cho biết sẽ khuyến nghị item nào cho họ.  
3) Duyệt theo từng item và cho biết sẽ khuyến nghị item này cho các user nào. Có nhận xét gì về kết quả thu được ở 2) và 3)?

In [20]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# --- Đọc dữ liệu ---
movies = pd.read_csv("movie_lens/movies.csv")
ratings = pd.read_csv("movie_lens/ratings.csv")

# --- Tạo ma trận user-movie ---
user_movie = ratings.pivot_table(
    index='userId',
    columns='movieId',
    values='rating'
)

print("📊 Ma trận đánh giá (user–movie):")
print(user_movie.head())


📊 Ma trận đánh giá (user–movie):
movieId  1       2       3       4       5       6       7       8       \
userId                                                                    
1           4.0     NaN     4.0     NaN     NaN     4.0     NaN     NaN   
2           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
3           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
4           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
5           4.0     NaN     NaN     NaN     NaN     NaN     NaN     NaN   

movieId  9       10      ...  193565  193567  193571  193573  193579  193581  \
userId                   ...                                                   
1           NaN     NaN  ...     NaN     NaN     NaN     NaN     NaN     NaN   
2           NaN     NaN  ...     NaN     NaN     NaN     NaN     NaN     NaN   
3           NaN     NaN  ...     NaN     NaN     NaN     NaN     NaN     NaN   
4           NaN     NaN  ...     NaN     

In [21]:
# Tính độ tương đồng giữa các user
user_similarity = pd.DataFrame(
    cosine_similarity(user_movie.fillna(0)),
    index=user_movie.index,
    columns=user_movie.index
)

# Tính độ tương đồng giữa các item (phim)
item_similarity = pd.DataFrame(
    cosine_similarity(user_movie.fillna(0).T),
    index=user_movie.columns,
    columns=user_movie.columns
)

print("\n🤝 Ma trận tương đồng giữa các user:")
print(user_similarity.round(2).head())

print("\n🎬 Ma trận tương đồng giữa các item:")
print(item_similarity.round(2).iloc[:5, :5])



🤝 Ma trận tương đồng giữa các user:
userId   1     2     3     4     5     6     7     8     9     10   ...   601  \
userId                                                              ...         
1       1.00  0.03  0.06  0.19  0.13  0.13  0.16  0.14  0.06  0.02  ...  0.08   
2       0.03  1.00  0.00  0.00  0.02  0.03  0.03  0.03  0.00  0.07  ...  0.20   
3       0.06  0.00  1.00  0.00  0.01  0.00  0.00  0.00  0.00  0.00  ...  0.01   
4       0.19  0.00  0.00  1.00  0.13  0.09  0.12  0.06  0.01  0.03  ...  0.09   
5       0.13  0.02  0.01  0.13  1.00  0.30  0.11  0.43  0.00  0.03  ...  0.07   

userId   602   603   604   605   606   607   608   609   610  
userId                                                        
1       0.16  0.22  0.07  0.15  0.16  0.27  0.29  0.09  0.15  
2       0.02  0.01  0.00  0.00  0.03  0.01  0.05  0.03  0.10  
3       0.00  0.02  0.00  0.01  0.01  0.02  0.02  0.00  0.03  
4       0.13  0.31  0.05  0.08  0.20  0.13  0.15  0.03  0.11  
5       0.42  0.1

In [22]:
def predict_user_based(user, item):
    if item not in user_movie.columns:
        return np.nan
    if not np.isnan(user_movie.loc[user, item]):
        return user_movie.loc[user, item]
    
    sim_scores = user_similarity.loc[user]
    ratings_for_item = user_movie[item]
    mask = ratings_for_item.notna()
    
    numerator = (sim_scores[mask] * ratings_for_item[mask]).sum()
    denominator = sim_scores[mask].abs().sum()
    
    return numerator / denominator if denominator != 0 else np.nan


In [23]:
def predict_item_based(user, item):
    if item not in user_movie.columns:
        return np.nan
    if not np.isnan(user_movie.loc[user, item]):
        return user_movie.loc[user, item]
    
    sim_scores = item_similarity.loc[item]
    ratings_by_user = user_movie.loc[user]
    mask = ratings_by_user.notna()
    
    numerator = (sim_scores[mask] * ratings_by_user[mask]).sum()
    denominator = sim_scores[mask].abs().sum()
    
    return numerator / denominator if denominator != 0 else np.nan


In [24]:
print("\n🎯 Khuyến nghị phim cho từng user (User-based CF):")
for user in user_movie.index[:5]:  # chỉ in 5 user đầu
    unseen = user_movie.loc[user][user_movie.loc[user].isna()]
    predictions = {i: predict_user_based(user, i) for i in unseen.index}
    if predictions:
        top_item = max(predictions, key=predictions.get)
        title = movies[movies.movieId == top_item]['title'].values[0]
        print(f"User {user} nên xem: 🎬 {title} (dự đoán {predictions[top_item]:.2f})")

print("\n🎯 Khuyến nghị user cho từng phim (Item-based CF):")
for item in user_movie.columns[:5]:  # chỉ in 5 phim đầu
    unseen_users = user_movie[item][user_movie[item].isna()].index
    predictions = {u: predict_item_based(u, item) for u in unseen_users}
    if predictions:
        top_user = max(predictions, key=predictions.get)
        print(f"Phim ID {item} nên gợi ý cho user {top_user} (dự đoán {predictions[top_user]:.2f})")



🎯 Khuyến nghị phim cho từng user (User-based CF):
User 1 nên xem: 🎬 Lesson Faust (1994) (dự đoán 5.00)
User 2 nên xem: 🎬 Awfully Big Adventure, An (1995) (dự đoán 5.00)
User 3 nên xem: 🎬 Heidi Fleiss: Hollywood Madam (1995) (dự đoán 5.00)
User 4 nên xem: 🎬 Jonah Who Will Be 25 in the Year 2000 (Jonas qui aura 25 ans en l'an 2000) (1976) (dự đoán 5.00)
User 5 nên xem: 🎬 Lamerica (1994) (dự đoán 5.00)

🎯 Khuyến nghị user cho từng phim (Item-based CF):
Phim ID 1 nên gợi ý cho user 53 (dự đoán 5.00)
Phim ID 2 nên gợi ý cho user 53 (dự đoán 5.00)
Phim ID 3 nên gợi ý cho user 53 (dự đoán 5.00)
Phim ID 4 nên gợi ý cho user 53 (dự đoán 5.00)
Phim ID 5 nên gợi ý cho user 53 (dự đoán 5.00)


# data.csv

In [25]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# ---------------------------
# 1️⃣ Đọc dữ liệu
# ---------------------------
# Giả sử file có 3 cột: userId, movieId, rating
du_lieu = pd.read_csv("data.csv", header=None, names=["userId", "movieId", "rating"])

print("📂 Dữ liệu ban đầu:")
print(du_lieu.head())

# ---------------------------
# 2️⃣ Tạo ma trận user–movie (utility matrix)
# ---------------------------
ma_tran = du_lieu.pivot_table(index="userId", columns="movieId", values="rating")

print("\n📊 Ma trận user–movie:")
print(ma_tran.head())

# ---------------------------
# 3️⃣ Tính ma trận tương đồng giữa user và giữa item
# ---------------------------
user_similarity = pd.DataFrame(
    cosine_similarity(ma_tran.fillna(0)),
    index=ma_tran.index,
    columns=ma_tran.index
)

item_similarity = pd.DataFrame(
    cosine_similarity(ma_tran.fillna(0).T),
    index=ma_tran.columns,
    columns=ma_tran.columns
)

print("\n🤝 Ma trận tương đồng giữa các user:")
print(user_similarity.round(2).head())

print("\n🎬 Ma trận tương đồng giữa các item:")
print(item_similarity.round(2).iloc[:5, :5])

# ---------------------------
# 4️⃣ Hàm dự đoán rating (User-based CF)
# ---------------------------
def du_doan_user_based(user, phim):
    if phim not in ma_tran.columns:
        return np.nan
    if not np.isnan(ma_tran.loc[user, phim]):
        return ma_tran.loc[user, phim]
    
    diem_tuong_dong = user_similarity.loc[user]
    diem_phim = ma_tran[phim]
    mask = diem_phim.notna()
    
    tu = (diem_tuong_dong[mask] * diem_phim[mask]).sum()
    mau = diem_tuong_dong[mask].abs().sum()
    
    return tu / mau if mau != 0 else np.nan

# ---------------------------
# 5️⃣ Hàm dự đoán rating (Item-based CF)
# ---------------------------
def du_doan_item_based(user, phim):
    if phim not in ma_tran.columns:
        return np.nan
    if not np.isnan(ma_tran.loc[user, phim]):
        return ma_tran.loc[user, phim]
    
    diem_tuong_dong = item_similarity.loc[phim]
    diem_user = ma_tran.loc[user]
    mask = diem_user.notna()
    
    tu = (diem_tuong_dong[mask] * diem_user[mask]).sum()
    mau = diem_tuong_dong[mask].abs().sum()
    
    return tu / mau if mau != 0 else np.nan

# ---------------------------
# 6️⃣ Khuyến nghị phim cho từng user (User-based)
# ---------------------------
print("\n🎯 Khuyến nghị phim cho từng user (User-based CF):")
for user in ma_tran.index[:5]:  # chỉ in 5 user đầu
    phim_chua_xem = ma_tran.loc[user][ma_tran.loc[user].isna()]
    du_doan = {p: du_doan_user_based(user, p) for p in phim_chua_xem.index}
    if du_doan:
        phim_goi_y = max(du_doan, key=du_doan.get)
        print(f"👤 User {user} → nên xem phim ID {phim_goi_y} (dự đoán: {du_doan[phim_goi_y]:.2f})")

# ---------------------------
# 7️⃣ Khuyến nghị user cho từng phim (Item-based)
# ---------------------------
print("\n🎬 Khuyến nghị user cho từng phim (Item-based CF):")
for phim in ma_tran.columns[:5]:  # chỉ in 5 phim đầu
    user_chua_xem = ma_tran[phim][ma_tran[phim].isna()].index
    du_doan = {u: du_doan_item_based(u, phim) for u in user_chua_xem}
    if du_doan:
        user_goi_y = max(du_doan, key=du_doan.get)
        print(f"🎥 Phim ID {phim} → nên gợi ý cho user {user_goi_y} (dự đoán: {du_doan[user_goi_y]:.2f})")

# ---------------------------
# 8️⃣ Nhận xét
# ---------------------------
print("""
🧠 Nhận xét:
- User-based CF: Dựa trên người có hành vi tương tự → tốt khi số lượng phim nhỏ, user nhiều.
- Item-based CF: Dựa trên phim có đánh giá tương tự → ổn định hơn khi dữ liệu thưa.
- Kết quả 2 cách thường gần giống nhau, nhưng item-based CF dễ mở rộng hơn (Netflix, Amazon dùng nhiều).
""")


📂 Dữ liệu ban đầu:
   userId  movieId  rating
0       1       18       5
1       3       12       4
2       3        9       3
3       3       15       1
4       3       12       2

📊 Ma trận user–movie:
movieId  1    2    3    4    5   6   7   8    9   10   11        12   13   14  \
userId                                                                          
1       NaN  NaN  NaN  NaN  NaN NaN NaN NaN  NaN NaN  NaN       NaN  NaN  NaN   
3       NaN  NaN  2.0  NaN  3.0 NaN NaN NaN  3.0 NaN  NaN  2.666667  NaN  NaN   
4       NaN  NaN  NaN  NaN  NaN NaN NaN NaN  NaN NaN  2.0       NaN  NaN  NaN   
5       NaN  2.0  NaN  NaN  NaN NaN NaN NaN  4.0 NaN  NaN  2.000000  3.0  5.0   
6       NaN  NaN  NaN  4.0  NaN NaN NaN NaN  NaN NaN  NaN       NaN  3.0  NaN   

movieId   15  16   17   18   19   20  
userId                                
1        NaN NaN  NaN  5.0  NaN  NaN  
3        1.0 NaN  1.0  3.0  NaN  NaN  
4        NaN NaN  NaN  3.0  1.0  NaN  
5        NaN NaN  NaN  NaN  NaN  

In [ ]:
+